In [ ]:
!pip install gradio

In [ ]:
!pip install sahi gradio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import gradio as gr
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.metrics.pairwise import cosine_similarity
import pickle

# Function to preprocess an image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Function to extract features from an image
def extract_features(img_path):
    img_array = preprocess_image(img_path)
    features = model.predict(img_array)
    return features.flatten()

# Function to compute and store features for the dataset
def compute_and_store_features(dataset_path, model, features_path):
    image_paths = []
    features_list = []

    for filename in os.listdir(dataset_path):
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".bmp"):
            img_path = os.path.join(dataset_path, filename)
            image_paths.append(img_path)
            features = extract_features(img_path)
            features_list.append(features)

    features_array = np.array(features_list)
    with open(features_path, 'wb') as f:
        pickle.dump({'image_paths': image_paths, 'features': features_array}, f)

# Function to load precomputed features
def load_features(features_path):
    with open(features_path, 'rb') as f:
        features_data = pickle.load(f)
    return features_data['image_paths'], features_data['features']

# Function to get similar images
def get_similar_images(query_image_path, dataset_path, features_path, num_similar_images=10):
    # Check if features are precomputed
    if os.path.exists(features_path):
        dataset_image_paths, dataset_features = load_features(features_path)
    else:
        # Compute and store features if not precomputed
        compute_and_store_features(dataset_path, model, features_path)
        dataset_image_paths, dataset_features = load_features(features_path)

    # Extract features from the query image using the loaded model
    query_features = extract_features(query_image_path)

    # Calculate cosine similarity between query and dataset images
    similarities = cosine_similarity(query_features.reshape(1, -1), dataset_features)[0]

    # Sort the images based on similarity
    sorted_indices = np.argsort(similarities)[::-1]

    # Get the top N similar images
    top_indices = sorted_indices[:num_similar_images]
    similar_images = [dataset_image_paths[i] for i in top_indices]

    return similar_images

# Load your pre-trained model
model = load_model('/content/drive/MyDrive/images dataset/model.h5')

# Path to the dataset
dataset_path = '/content/drive/MyDrive/images dataset/Whole dataset'

# Path to save and load the features
features_path = '/content/drive/MyDrive/images dataset/features.pkl'

# Gradio interface setup
def get_and_display_similar_images(query_image):
    # Convert the NumPy array to an image for display
    plt.imshow(query_image)
    plt.axis('off')
    plt.show()

    # Save the query image to a file
    query_image_path = 'query_image.jpg'
    plt.savefig(query_image_path)

    # Get similar images
    similar_images = get_similar_images(query_image_path, dataset_path, features_path)
    # Return the list of similar image paths for display
    return [gr.Image(img_path) for img_path in similar_images]

# Gradio interface for taking an image as input and displaying similar images
iface = gr.Interface(
    fn=get_and_display_similar_images,
    inputs=gr.Image(),
    outputs=[gr.Image() for _ in range(10)],  # Gradio will automatically display the list of 10 images
)



iface.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://3bae0d5e71e6973ca4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 1s 873ms/step
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3bae0d5e71e6973ca4.gradio.live
